In [2]:
pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report


In [3]:
# Define a function to extract features from audio files
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=27)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y=y)
    rms = librosa.feature.rms(y=y)

 # Concatenate all features and average them across time
    features = np.hstack([
        np.mean(mfcc.T, axis=0),
        np.mean(chroma.T, axis=0),
        np.mean(zcr.T, axis=0),
        np.mean(rms.T, axis=0)
    ])
    return features

In [6]:

# Load your data (path to your Kaggle dataset)
screaming_files = [os.path.join('Screaming', f) for f in os.listdir('Screaming') if f.endswith('.wav')]
not_screaming_files = [os.path.join('NotScreaming', f) for f in os.listdir('NotScreaming') if f.endswith('.wav')]

# Prepare the dataset
X = []
y = []

In [7]:
# Extract features from screaming samples
for file in screaming_files:
    features = extract_features(file)
    X.append(features)
    y.append(1)  # Screaming class

In [8]:
# Extract features from not-screaming samples
for file in not_screaming_files:
    features = extract_features(file)
    X.append(features)
    y.append(0)  # Not-screaming class

C:\Users\bharg\AppData\Roaming\Python\Python312\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [9]:

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

In [11]:
# Handle class imbalance using SMOTE (Synthetic Minority Over-sampling Technique)
from collections import Counter

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Resampled class distribution:", Counter(y_resampled))


Resampled class distribution: Counter({np.int64(1): 2631, np.int64(0): 2631})


In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [13]:
# Scale the features (important for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Train the RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
model.fit(X_train_scaled, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [15]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy on test set: 0.8841405508072174
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88       508
           1       0.91      0.87      0.89       545

    accuracy                           0.88      1053
   macro avg       0.88      0.88      0.88      1053
weighted avg       0.89      0.88      0.88      1053



In [16]:

# Save the trained model
joblib.dump(model, 'scream_detection_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [17]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)
print("Best parameters:", grid_search.best_params_)


Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
